In [2]:
#coding=utf-8
import requests, hashlib, urlparse, time, string, base64, sys, threading
from bs4 import BeautifulSoup
data = ''
cookies = 0


In [3]:
def url_decode(url):
    #url = "https://reg.ntuh.gov.tw/WebAdministration/RegistForm.aspx?newx=UwBlAHIAdgBpAGMAZQBJAEQAUwBFAD0ANAAwADkANQAxADgAOQAmAEUAbgBjAHIAeQBwAHQAQwBvAGQAZQA9AFQAMABTAFUAUgBHADEAMwAyADAAMgAwADAAMwAyADAAJgB1AHMAZQBEAHIAUgBlAHMAdABDAG4AdAA9AG4A0"
    url_p = urlparse.urlparse(url).query[5:]
    print "url_p:", url_p
    print "----------------------------------------"
    
    if url_p[-1:] == '0':
        parameter = base64.b64decode(url_p[:-1]).decode('utf-8')
        #print "parameter:",parameter
        #print "parameter:", parameter.split('&')
        parameter = parameter.encode('base64')+'0'
        new_url = "https://reg.ntuh.gov.tw/WebAdministration/RegistForm.aspx?newx=" + parameter
        print "\nThis URL is:\n", new_url.replace('\n','')
        print "----------------------------------------\n"
    if url_p[-1:] == '2':
        parameter = base64.b64decode(url_p+"=").decode('utf-8')
        #print "parameter:",parameter
        #print "parameter:", parameter.split('&')
        parameter = parameter.encode('base64')+'0'
        new_url = "https://reg.ntuh.gov.tw/WebAdministration/RegistForm.aspx?newx=" + parameter
        print "\nThis URL is:\n", new_url.replace('\n','')
        print "----------------------------------------\n"
    
    
    for j in range(1):
            r = requests.get(new_url.replace('\n',''), data = data, cookies=cookies)
            soup = BeautifulSoup(r.text, "html.parser")
            #print soup.text
            if soup.find(id="ShowTime").text:
                print "Web Check:\n", soup.find(id="ShowTime").text, "\n", new_url.replace('\n','')
                print "----------------------------------------\n"
                break
            else:
                continue



In [76]:
def url_remake(url):
    url_p = urlparse.urlparse(url).query[5:]
    if url_p[-1:] == '0':
        parameter = base64.b64decode(url_p[:-1])
        print "parameter:",parameter.replace('\x00','')
        print "parameter:",parameter.replace('\x00','').split('&')
    if url_p[-1:] == '2':
        parameter = base64.b64decode(url_p+"=").decode('utf-8')
        print "parameter:",parameter.replace('\x00','')
        print "parameter:",parameter.replace('\x00','').split('&')
        print "parameter:",parameter.split('&')
    print "----------------------------------------\n"
   

    ServiceIDSE = int(parameter.replace('\x00','').split('&')[0][12:],)
    EncryptCode = str(parameter.replace('\x00','').split('&')[1][12:],)
    useDrRestCnt = str(parameter.replace('\x00','').split('&')[2][13:],)
    print "ServiceIDSE:", ServiceIDSE
    print "EncryptCode:", EncryptCode[:8] + dateyouwant
    print "useDrRestCnt:", useDrRestCnt[0]
    
    oldIDSE = ServiceIDSE
    ServiceIDSE+=ServiceIDSE_guess ### Gap Number
    

    #Bruteforce
    for i in xrange(1,10000):
        ServiceIDSE = ServiceIDSE+1
        parameter = "ServiceIDSE=" + str(ServiceIDSE) + "&EncryptCode=" +  (EncryptCode[:8] + dateyouwant) + "&useDrRestCnt=" + useDrRestCnt[0]
        parameter = "\x00".join(parameter)
        parameter += "\x00\r\n\r" 
        parameter_detail = parameter
        parameter = base64.b64encode(parameter).replace('0=','2')+'0'
        #print "parameter_b64:", parameter    
        new_url = "https://reg.ntuh.gov.tw/WebAdministration/RegistForm.aspx?newx=" + parameter
        #print "\n",new_url.replace('\n','')

        time.sleep(0.1)
        #sys.stdout.write("\r%d%%" % i)
        sys.stdout.write("\rServiceIDSE= %s" % ServiceIDSE)
        sys.stdout.flush()
        
        for j in range(1):
            requests.adapters.DEFAULT_RETRIES = 5 #
            s = requests.session()
            s.keep_alive = False #
            r = requests.get(new_url.replace('\n',''), data = data, cookies=cookies)
            soup = BeautifulSoup(r.text, "html.parser")
            #print soup.text
            if r.status_code==500:
                print "\n Gap =",ServiceIDSE-oldIDSE,"500 error"
                break
            elif soup.find(id="ShowTime").text:
                print "\n###################Successful######################" 
                print soup.find(id="ShowTime").text
                print " Parameter:", parameter_detail,"\n Gap =",ServiceIDSE-oldIDSE
                print "Next URL is:", new_url.replace('\n','')
                print "\n###################################################" 
                return
            

In [ ]:
#url=str(input('請輸入想掛的醫生網址：'))
url="https://reg.ntuh.gov.tw/webadministration/RegistForm.aspx?newx=UwBlAHIAdgBpAGMAZQBJAEQAUwBFAD0ANAAxADEAOAA0ADQAOAAmAEUAbgBjAHIAeQBwAHQAQwBvAGQAZQA9AFQAMABEAEUATgBUADAAOQAyADAAMgAwADAANAAwADgAJgB1AHMAZQBEAHIAUgBlAHMAdABDAG4AdAA9AA2"
url_decode(url)
dateyouwant=str(input('請輸入想掛號的日期：'))
ServiceIDSE_guess = int(input('Guess Gap Number (Min=0):'))
url_remake(url)


In [70]:
#url= "https://reg.ntuh.gov.tw/WebAdministration/RegistForm.aspx?newx=UwBlAHIAdgBpAGMAZQBJAEQAUwBFAD0ANAAxADQANgAyADkAOQAmAEUAbgBjAHIAeQBwAHQAQwBvAGQAZQA9AFQAMABEAEUATgBUADIAMAAyADAAMgAwADAANQAwADEAJgB1AHMAZQBEAHIAUgBlAHMAdABDAG4AdAA9AG4A0"
url="https://reg.ntuh.gov.tw/WebAdministration/RegistForm.aspx?newx=UwBlAHIAdgBpAGMAZQBJAEQAUwBFAD0ANAAxADIANgAwADIANgAmAEUAbgBjAHIAeQBwAHQAQwBvAGQAZQA9AFQAMABEAEUATgBUADAAOQAyADAAMgAwADAANAAxADUAJgB1AHMAZQBEAHIAUgBlAHMAdABDAG4AdAA9AA0ADQoN0"
dateyouwant=str(input('請輸入想掛號的日期：'))
ServiceIDSE_guess = int(input('Guess Gap Number (Min=0):'))
url_remake(url)


請輸入想掛號的日期：20200422
Guess Gap Number (Min=0):8370
parameter: ServiceIDSE=4126026&EncryptCode=T0DENT0920200415&useDrRestCnt=

parameter: ['ServiceIDSE=4126026', 'EncryptCode=T0DENT0920200415', 'useDrRestCnt=\r\r\n\r']
----------------------------------------

ServiceIDSE: 4126026
EncryptCode: T0DENT0920200422
useDrRestCnt: 
ServiceIDSE= 4134406
###################Successful######################
109.4.22 星期三 下午
 arameter: S e r v i c e I D S E = 4 1 3 4 4 0 6 & E n c r y p t C o d e = T 0 D E N T 0 9 2 0 2 0 0 4 2 2 & u s e D r R e s t C n t = 

Next URL is: https://reg.ntuh.gov.tw/WebAdministration/RegistForm.aspx?newx=UwBlAHIAdgBpAGMAZQBJAEQAUwBFAD0ANAAxADMANAA0ADAANgAmAEUAbgBjAHIAeQBwAHQAQwBvAGQAZQA9AFQAMABEAEUATgBUADAAOQAyADAAMgAwADAANAAyADIAJgB1AHMAZQBEAHIAUgBlAHMAdABDAG4AdAA9AA0ADQoN0

###################################################


In [71]:
url="https://reg.ntuh.gov.tw/WebAdministration/RegistForm.aspx?newx=UwBlAHIAdgBpAGMAZQBJAEQAUwBFAD0ANAAxADMANAA0ADAANgAmAEUAbgBjAHIAeQBwAHQAQwBvAGQAZQA9AFQAMABEAEUATgBUADAAOQAyADAAMgAwADAANAAyADIAJgB1AHMAZQBEAHIAUgBlAHMAdABDAG4AdAA9AA0ADQoN0"
dateyouwant=str(input('請輸入想掛號的日期：'))
ServiceIDSE_guess = int(input('Guess Gap Number (Min=0):'))
url_remake(url)

請輸入想掛號的日期：20200429
Guess Gap Number (Min=0):8300
parameter: ServiceIDSE=4134406&EncryptCode=T0DENT0920200422&useDrRestCnt=

parameter: ['ServiceIDSE=4134406', 'EncryptCode=T0DENT0920200422', 'useDrRestCnt=\r\r\n\r']
----------------------------------------

ServiceIDSE: 4134406
EncryptCode: T0DENT0920200429
useDrRestCnt: 
ServiceIDSE= 4142732
 i = 26 500 error
ServiceIDSE= 4142771
 i = 65 500 error
ServiceIDSE= 4142783
 i = 77 500 error
ServiceIDSE= 4142791
 i = 85 500 error
ServiceIDSE= 4142864
 i = 158 500 error
ServiceIDSE= 4142887
 i = 181 500 error
ServiceIDSE= 4142931
 i = 225 500 error
ServiceIDSE= 4142932
 i = 226 500 error
ServiceIDSE= 4142959
 i = 253 500 error
ServiceIDSE= 4142988
 i = 282 500 error
ServiceIDSE= 4143012
###################Successful######################
109.4.29 星期三 下午
 arameter: S e r v i c e I D S E = 4 1 4 3 0 1 2 & E n c r y p t C o d e = T 0 D E N T 0 9 2 0 2 0 0 4 2 9 & u s e D r R e s t C n t = 

Next URL is: https://reg.ntuh.gov.tw/WebAdministration

In [74]:
url="https://reg.ntuh.gov.tw/WebAdministration/RegistForm.aspx?newx=UwBlAHIAdgBpAGMAZQBJAEQAUwBFAD0ANAAxADQAMwAwADEAMgAmAEUAbgBjAHIAeQBwAHQAQwBvAGQAZQA9AFQAMABEAEUATgBUADAAOQAyADAAMgAwADAANAAyADkAJgB1AHMAZQBEAHIAUgBlAHMAdABDAG4AdAA9AA0ADQoN0"
dateyouwant=str(input('請輸入想掛號的日期：'))
ServiceIDSE_guess = int(input('Guess Gap Number (Min=0):'))
url_remake(url)

請輸入想掛號的日期：20200506
Guess Gap Number (Min=0):8300
parameter: ServiceIDSE=4143012&EncryptCode=T0DENT0920200429&useDrRestCnt=

parameter: ['ServiceIDSE=4143012', 'EncryptCode=T0DENT0920200429', 'useDrRestCnt=\r\r\n\r']
----------------------------------------

ServiceIDSE: 4143012
EncryptCode: T0DENT0920200506
useDrRestCnt: 
ServiceIDSE= 4151340
 Gap = 8328 500 error
ServiceIDSE= 4151379
 Gap = 8367 500 error
ServiceIDSE= 4151391
 Gap = 8379 500 error
ServiceIDSE= 4151399
 Gap = 8387 500 error
ServiceIDSE= 4151472
 Gap = 8460 500 error
ServiceIDSE= 4151495
 Gap = 8483 500 error
ServiceIDSE= 4151539
 Gap = 8527 500 error
ServiceIDSE= 4151540
 Gap = 8528 500 error
ServiceIDSE= 4151567
 Gap = 8555 500 error
ServiceIDSE= 4151596
 Gap = 8584 500 error
ServiceIDSE= 4151620
###################Successful######################
109.5.6 星期三 下午
 arameter: S e r v i c e I D S E = 4 1 5 1 6 2 0 & E n c r y p t C o d e = T 0 D E N T 0 9 2 0 2 0 0 5 0 6 & u s e D r R e s t C n t = 

Next URL is: https://

In [80]:
url="https://reg.ntuh.gov.tw/WebAdministration/RegistForm.aspx?newx=UwBlAHIAdgBpAGMAZQBJAEQAUwBFAD0ANAAxADUAMwA4ADEAOQAmAEUAbgBjAHIAeQBwAHQAQwBvAGQAZQA9AFQAMABEAEUATgBUADUAOQAyADAAMgAwADAANQAwADcAJgB1AHMAZQBEAHIAUgBlAHMAdABDAG4AdAA9AG4A0"
dateyouwant=str(input('請輸入想掛號的日期：'))
ServiceIDSE_guess = int(input('Guess Gap Number (Min=0):'))
url_remake(url)

請輸入想掛號的日期：20200514
Guess Gap Number (Min=0):0
parameter: ServiceIDSE=4153819&EncryptCode=T0DENT5920200507&useDrRestCnt=n
parameter: ['ServiceIDSE=4153819', 'EncryptCode=T0DENT5920200507', 'useDrRestCnt=n']
----------------------------------------

ServiceIDSE: 4153819
EncryptCode: T0DENT5920200514
useDrRestCnt: n
ServiceIDSE= 4153875
 Gap = 56 500 error
ServiceIDSE= 4153876
 Gap = 57 500 error
ServiceIDSE= 4153976
 Gap = 157 500 error
ServiceIDSE= 4153977
 Gap = 158 500 error
ServiceIDSE= 4153978
 Gap = 159 500 error
ServiceIDSE= 4153979
 Gap = 160 500 error
ServiceIDSE= 4153980
 Gap = 161 500 error
ServiceIDSE= 4153981
 Gap = 162 500 error
ServiceIDSE= 4153982
 Gap = 163 500 error
ServiceIDSE= 4153983
 Gap = 164 500 error
ServiceIDSE= 4153984
 Gap = 165 500 error
ServiceIDSE= 4153985
 Gap = 166 500 error
ServiceIDSE= 4153986
 Gap = 167 500 error
ServiceIDSE= 4153987
 Gap = 168 500 error
ServiceIDSE= 4153988
 Gap = 169 500 error
ServiceIDSE= 4153989
 Gap = 170 500 error
ServiceIDSE= 41

ServiceIDSE= 4154598
 Gap = 779 500 error
ServiceIDSE= 4154644
 Gap = 825 500 error
ServiceIDSE= 4154646
 Gap = 827 500 error
ServiceIDSE= 4154647
 Gap = 828 500 error
ServiceIDSE= 4154674

KeyboardInterrupt: 

In [6]:
dateuwant = str(input("請輸入想掛號日期(ex.20200508):"))
url="https://reg.ntuh.gov.tw/WebAdministration/DeptServiceListUnderSameDateAMPM.aspx?x=RABlAHAAdABDAG8AZABlAD0ARABFAE4AVAAmAEgAbwBzAHAAQwBvAGQAZQA9AFQAMAAmAEEATQBQAE0AQwBvAGQAZQA9ADEAJgBEAGEAdABlAFMAdAByAGkAbgBnAD0AMgAwADIAMAAwADUAMAAxACYAUwB1AGIARABlAHAAdABDAG8AZABlAD0AMAAxAA2"
url_p = urlparse.urlparse(url).query[2:]
print "url_p:", url_p
print "----------------------------------------"
    
if url_p[-1:] == '0':
    parameter = base64.b64decode(url_p[:-1]).decode('utf-8')
    print "parameter:",parameter
    print "parameter:", parameter.split('&')
    parameter = parameter.encode('base64')+'0'
    new_url = "https://reg.ntuh.gov.tw/WebAdministration/RegistForm.aspx?newx=" + parameter
    print "\nThis URL is:\n", new_url.replace('\n','')
    print "----------------------------------------\n"
if url_p[-1:] == '2':
    parameter = base64.b64decode(url_p+"=").decode('utf-8')
    print "parameter:",parameter
    print "parameter:", parameter.split('&')

    DeptCode = str(parameter.replace('\x00','').split('&')[0][9:],)
    print "\nDeptCode:", DeptCode
    
    HospCode = str(parameter.replace('\x00','').split('&')[1][9:],)
    print "HospCode:", HospCode
    
    AMPMCode = str(parameter.replace('\x00','').split('&')[2][9:],)
    print "AMPMCode:", AMPMCode
    
    DateString = str(parameter.replace('\x00','').split('&')[3][11:],)
    print "DateString:", DateString
    
    SubDeptCode = str(parameter.replace('\x00','').split('&')[4][12:],)
    print "SubDeptCode:", SubDeptCode
    
    
    parameter = "DeptCode=" + str(DeptCode) + "&HospCode=" +  (HospCode[:8]) + "&AMPMCode=" + AMPMCode[0] + "&DateString=" + dateuwant + "&SubDeptCode=" + SubDeptCode
    parameter = "\x00".join(parameter)
    parameter += "\x00\r\n\r" 
    print "\n parameter =", parameter 
    parameter_detail = parameter
    parameter = base64.b64encode(parameter).replace('=','0')+'0'
    #print "parameter_b64:", parameter    
    new_url = "https://reg.ntuh.gov.tw/WebAdministration/DeptServiceListUnderSameDateAMPM.aspx?x=" + parameter
    print new_url

請輸入想掛號日期(ex.20200508):20200508
 url_p: RABlAHAAdABDAG8AZABlAD0ARABFAE4AVAAmAEgAbwBzAHAAQwBvAGQAZQA9AFQAMAAmAEEATQBQAE0AQwBvAGQAZQA9ADEAJgBEAGEAdABlAFMAdAByAGkAbgBnAD0AMgAwADIAMAAwADUAMAAxACYAUwB1AGIARABlAHAAdABDAG8AZABlAD0AMAAxAA2
----------------------------------------
parameter: D e p t C o d e = D E N T & H o s p C o d e = T 0 & A M P M C o d e = 1 & D a t e S t r i n g = 2 0 2 0 0 5 0 1 & S u b D e p t C o d e = 0 1 
parameter: [u'D\x00e\x00p\x00t\x00C\x00o\x00d\x00e\x00=\x00D\x00E\x00N\x00T\x00', u'\x00H\x00o\x00s\x00p\x00C\x00o\x00d\x00e\x00=\x00T\x000\x00', u'\x00A\x00M\x00P\x00M\x00C\x00o\x00d\x00e\x00=\x001\x00', u'\x00D\x00a\x00t\x00e\x00S\x00t\x00r\x00i\x00n\x00g\x00=\x002\x000\x002\x000\x000\x005\x000\x001\x00', u'\x00S\x00u\x00b\x00D\x00e\x00p\x00t\x00C\x00o\x00d\x00e\x00=\x000\x001\x00\r']

DeptCode: DENT
HospCode: T0
AMPMCode: 1
DateString: 20200501
SubDeptCode: 01

 parameter = D e p t C o d e = D E N T & H o s p C o d e = T 0 & A M P M C o d e = 1 & D a t e S t r i n 